# 비선형 회귀 모델

In [37]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [38]:
# 사용자 정의 데이터세트 선언
class CustomDataset(Dataset): # pytorch의 Dataset 클래스를 상속; DataLoader가 이 클래스의 __len__과 __getitem__을 사용
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x = df.iloc[:, 0].values
    self.y = df.iloc[:, 1].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x[index] ** 2, self.x[index]]) # (x^2, x) 형태의 새로운 피처을 만든다.
    y = torch.FloatTensor([self.y[index]])
    return x, y

  def __len__(self):
    return self.length

In [39]:
# 사용자 정의 모델 선언
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(2, 1) # 2개의 입력 노드, 1개의 출력 노드

  def forward(self, x):
    x = self.layer(x)
    return x

In [40]:
# 사용자 정의 데이터세트와 데이터로더
train_dataset = CustomDataset("/content/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [41]:
# GPU 연산 적용
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

모델과 손실 함수뿐만 아니라, 학습 시 모델에 입력하는 데이터 텐서(입력 x, 정답 y) 또한 반드시 동일한 장치로 보내주어야 한다.

In [42]:
# 학습 진행
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost += loss

  cost = cost / len(train_dataloader) # 평균값

  if (epoch + 1) % 1000 == 0:
      print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1004, -1.7005]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4779], device='cuda:0', requires_grad=True)], Cost : 0.074
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1001, -1.7033]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4793], device='cuda:0', requires_grad=True)], Cost : 0.080
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1002, -1.7033]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4805], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1000, -1.7034]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4818], device='cuda:0', requires_grad=True)], Cost : 0.081
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0998, -1.7035]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4828], device='cuda:0', requires_grad=True)]

결과:  
y = 3.1069x^2 - 1.7029x + 0.0571

In [43]:
# 모델 평가
with torch.no_grad():         # 그래디언트 계산을 비활성화 한다(가중치 업데이트 x).
  model.eval()                # 모델을 평가 모드로 전환
  inputs = torch.FloatTensor( # 테스트 데이터의 차원은 훈련 데이터와 동일한 구조여야 한다.
      [
          [1 ** 2, 1],
          [5 ** 2, 5],
          [11 ** 2, 11]
      ]
  ).to(device)
  outputs = model(inputs)
  print(outputs)

tensor([[  1.8839],
        [ 69.4761],
        [356.8772]], device='cuda:0')


no_grad(): 자동 미분을 하지 않도록 설정(기울기 계산 x)  

- 메모리 사용량을 줄여 추론에 적합한 상태로 전환  

model.eval(): 모델을 평가 모드로 전환  

- 이 설정을 하지 않으면, 동일한 입력에 대해서도 드롭아웃 때문에 예측 결과가 달라지는 등 일관성 없는 결과를 얻을 수 있다.  
- 다시 모델을 학습시킬 때는 반드시 model.train()을 호출하여 학습 모드로 되돌려야 한다.

In [44]:
# 모델 저장
torch.save(
    model,
    "/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/model.pt"
)

In [ ]:
# 모델 저장(파라미터만 저장)
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/model_state_dict.pt"
)

# 데이터세트 분리
train, validation, test 데이터를 분리해서 효율적으로 모델을 관리할 수 있다.

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

# 1. 사용자 정의 데이터세트 선언
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

# 2. 사용자 정의 모델 선언
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

# 3. 데이터세트 분리
dataset = CustomDataset("/content/non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

# 4. GPU 연산 적용
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

# 5. 학습 진행
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

# 6. 모델 평가
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20
Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0997, -1.7043]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5123], device='cuda:0', requires_grad=True)], Cost : 0.079
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7043]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4885], device='cuda:0', requires_grad=True)], Cost : 0.080
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.0993, -1.7043]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4787], device='cuda:0', requires_grad=True)], Cost : 0.076
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1001, -1.7041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4747], device='cuda:0', requires_grad=True)], Cost : 0.078
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7043]], device='cuda:0', requires_grad=True), Pa